In [2]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

In [3]:
df_fighters = pd.read_csv('fighters.csv')
df_cards = pd.read_csv('fightcards.csv')

## Initial Exploration

In [4]:
df_fighters.head()

,DoB,SApM,SLpM,height,name,reach,record,stance,strAcc,strDef,subAvg,tdAcc,tdAvg,tdDef,weight
0,Jan 29 1993,0.00,0.00,--,Gabriel Arges,NaN,0-0-0,NaN,0%,0%,0.0,0%,0.0,0%,170
1,--,0.00,0.00,--,Bobby Escalante,NaN,5-6-0,NaN,0%,0%,0.0,0%,0.0,0%,130
2,Nov 15 1991,5.17,2.50,"6' 2""",Ivan Erslan,72.0,14-5-0 (1 NC),Orthodox,44%,55%,0.0,20%,0.5,64%,205
3,--,0.00,0.00,"6' 1""",Houston Dorr,NaN,1-2-0,Orthodox,0%,0%,0.0,0%,0.0,0%,210
4,May 24 1978,1.67,1.86,"5' 8""",Paulo Filho,NaN,23-6-3,Southpaw,48%,66%,1.6,32%,2.4,42%,185


In [5]:
df_cards.head()

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,f2_td_succeed,fight_date,fights_location,round_format,round_fought,weight_class,winner,winning_method
0,UFC 289: Nunes vs. Aldana,Diana Belbita,50%,208,2,1,Maria Oliveira,38%,168,5,2,June 10 2023,"Vancouver, British Columbia, Canada",3,3,Women's Strawweight,Diana Belbita,U-DEC
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,68%,22,1,0,Bruna Brasil,58%,93,3,3,April 15 2023,"Kansas City, Missouri, USA",3,2,Women's Strawweight,Denise Gomes,KO/TKO
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,56%,91,3,0,Lando Vannata,37%,207,1,0,April 15 2023,"Kansas City, Missouri, USA",3,3,Lightweight,Daniel Zellhuber,U-DEC
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,31%,16,0,0,Tanner Boser,75%,40,1,1,April 15 2023,"Kansas City, Missouri, USA",3,1,Light Heavyweight,Ion Cutelaba,KO/TKO
4,UFC Fight Night: Holloway vs. Allen,Azamat Murzakanov,48%,131,6,1,Dustin Jacoby,52%,127,5,1,April 15 2023,"Kansas City, Missouri, USA",3,3,Light Heavyweight,Azamat Murzakanov,U-DEC


In [6]:
df_fighters.dtypes

DoB        object
SApM      float64
SLpM      float64
height     object
name       object
reach     float64
record     object
stance     object
strAcc     object
strDef     object
subAvg    float64
tdAcc      object
tdAvg     float64
tdDef      object
weight     object
dtype: object

In [7]:
df_cards.dtypes

card_name              object
f1                     object
f1_sig_strike_per      object
f1_sig_strike_total     int64
f1_td_attempt           int64
f1_td_succeed           int64
f2                     object
f2_sig_strike_per      object
f2_sig_strike_total     int64
f2_td_attempt           int64
f2_td_succeed           int64
fight_date             object
fights_location        object
round_format            int64
round_fought            int64
weight_class           object
winner                 object
winning_method         object
dtype: object

In [8]:
df_cards.isnull().sum()

card_name              0
f1                     0
f1_sig_strike_per      0
f1_sig_strike_total    0
f1_td_attempt          0
f1_td_succeed          0
f2                     0
f2_sig_strike_per      0
f2_sig_strike_total    0
f2_td_attempt          0
f2_td_succeed          0
fight_date             0
fights_location        0
round_format           0
round_fought           0
weight_class           0
winner                 0
winning_method         0
dtype: int64

In [9]:
df_fighters.isnull().sum()

DoB          0
SApM         0
SLpM         0
height       0
name         0
reach     2001
record       0
stance     901
strAcc       0
strDef       0
subAvg       0
tdAcc        0
tdAvg        0
tdDef        0
weight       0
dtype: int64

In [10]:
print(df_fighters.apply(lambda col: col.nunique()))

DoB       3057
SApM       815
SLpM       701
height      28
name      4379
reach       27
record    1244
stance       5
strAcc      83
strDef      85
subAvg      98
tdAcc       84
tdAvg      570
tdDef       93
weight     114
dtype: int64


In [11]:
# check for fighter with the same name
df_fighters[df_fighters.duplicated(subset='name', keep=False)]

,DoB,SApM,SLpM,height,name,reach,record,stance,strAcc,strDef,subAvg,tdAcc,tdAvg,tdDef,weight
353,Jan 15 1991,2.76,2.69,"5' 9""",Michael McDonald,70.0,17-4-0,Orthodox,42%,57%,1.4,66%,1.09,52%,135
442,Feb 06 1965,0.40,0.00,"5' 11""",Michael McDonald,NaN,1-1-0,Orthodox,0%,50%,0.0,0%,0.00,0%,205
1344,May 02 1983,3.67,4.00,"6' 2""",Tony Johnson,76.0,7-2-0,Orthodox,92%,22%,0.0,0%,0.00,90%,205
1442,--,4.73,2.00,"6' 1""",Tony Johnson,NaN,11-3-0,NaN,53%,31%,0.0,22%,2.00,0%,265
2245,Jul 21 1986,4.46,2.44,"5' 10""",Joey Gomez,73.0,6-2-0,Orthodox,28%,55%,0.0,100%,0.62,50%,135
2448,Aug 29 1989,3.33,3.73,"5' 10""",Joey Gomez,71.0,7-1-0,Orthodox,49%,50%,0.0,28%,2.00,0%,155
2594,--,0.00,0.00,--,Mike Davis,NaN,2-0-0,NaN,0%,0%,0.0,0%,0.00,0%,--
2603,Oct 07 1992,5.23,4.73,"6' 0""",Mike Davis,72.0,12-3-0,Orthodox,53%,54%,0.8,44%,2.40,60%,155
3210,Dec 27 1996,4.13,4.87,"5' 7""",Jean Silva,69.0,16-2-0,Orthodox,52%,56%,0.9,50%,0.60,86%,145
3373,Mar 16 1990,4.81,3.83,"5' 4""",Bruno Silva,65.0,14-7-2 (1 NC),Orthodox,49%,50%,0.2,23%,1.79,60%,125


## Data cleaning

#### df_fighters cleaning

Quite a lot of preprocessing will need to be done here. First, a few fighters have the same name there. We will add elements to their name to distinguish them. The easiest way would be by adding their weights, since fortunately, the fighters with the same name in our list here have different weight class. We also have 2 Mike Davis, with the 2nd one missing a lot of data. therefore, we will drop him altogether

In [12]:
# change certain name so that we don't have duplication
df_fighters.iloc[446, 4] = "Michael McDonald 135"
df_fighters.iloc[1318, 4] = "Tony Johnson 265"
df_fighters.iloc[2092, 4] = "Joey Gomez 155"
df_fighters.iloc[3300, 4] = "Bruno Silva 185"
df_fighters.drop([2404], inplace=True)

We can see that in both of the table, there are a lot of percentages that are in string. We will therefore create a function to convert those to decimal

In [13]:
def p2d(df, columns):
    for column in columns:
        df[column] = df[column].str.strip('%')
        df[column] = pd.to_numeric(df[column]) / 100


In [14]:
p2d(df_fighters, ['strAcc', 'strDef', 'tdAcc', 'tdDef'])


In [15]:
df_fighters.columns

Index(['DoB', 'SApM', 'SLpM', 'height', 'name', 'reach', 'record', 'stance',
       'strAcc', 'strDef', 'subAvg', 'tdAcc', 'tdAvg', 'tdDef', 'weight'],
      dtype='object')

In [16]:

# Some fighters do not have statistics available, and we will remove those fighters.
fighters_clean = df_fighters.loc[~(
                               (df_fighters["strDef"] == 0) &
                               (df_fighters["tdAvg"] == 0) &
                               (df_fighters["tdAcc"] == 0) &
                               (df_fighters["tdDef"] == 0) &
                               (df_fighters["subAvg"] == 0))].copy()

In [17]:
per_missing_dob = fighters_clean[fighters_clean['DoB'] == '--']['DoB'].count() / len(fighters_clean)
print('the percentage of missing Date of Birth is: {:.2f}'.format(per_missing_dob))

the percentage of missing Date of Birth is: 0.09


Although the percentage of missing date of birth is quite high, we would remove these fighters anyways since the lack of birth date often mean that the fighter only fought 1 match in UFC, and was quite irrelevant to the sport. Also, it often means that these fighters fought in the very early days of ufc. And as the fan knows, the sport has changed dramatically since then. Including these fighters might add more random noise to our dataset than helping

In [18]:
fighters_clean = fighters_clean[~(fighters_clean['DoB'] == '--')].copy()

In [19]:
print("Initially, there are {} fighers in total, after clean up: {} fighers".format(len(df_fighters), len(fighters_clean)))

Initially, there are 4384 fighers in total, after clean up: 3274 fighers


In [20]:
def get_birth_year(dob):
    return datetime.strptime(dob, '%b %d %Y').year

fighters_clean['born_year'] = fighters_clean['DoB'].apply(lambda x: get_birth_year(x))
fighters_clean.drop(['DoB'], inplace=True, axis=1)

In [21]:
fighters_clean.set_index('name', inplace=True)

In [22]:
fighters_clean.head()

,SApM,SLpM,height,reach,record,stance,strAcc,strDef,subAvg,tdAcc,tdAvg,tdDef,weight,born_year
name,,,,,,,,,,,,,,
Ivan Erslan,5.17,2.50,"6' 2""",72.0,14-5-0 (1 NC),Orthodox,0.44,0.55,0.0,0.20,0.50,0.64,205,1991
Paulo Filho,1.67,1.86,"5' 8""",NaN,23-6-3,Southpaw,0.48,0.66,1.6,0.32,2.40,0.42,185,1978
Paul Bradley,4.13,2.26,"5' 9""",NaN,23-8-0 (2 NC),Orthodox,0.32,0.56,0.0,0.16,1.12,1.00,170,1983
Dan Ige,3.56,3.64,"5' 7""",71.0,19-10-0,Orthodox,0.45,0.56,0.2,0.25,0.89,0.56,145,1991
DaMarques Johnson,3.75,2.11,"6' 1""",75.0,15-15-0,Orthodox,0.37,0.51,0.7,0.55,1.18,0.19,170,1982


From my experience, reach is 1 of the most important factor that would determine the strategy and outcome of a fight. yet as we see in the first look of the data, the data is missing for more than a thoudsand of fighters. 

The method we will use to move forward is as follow: Reach is often a function of height (although not perfect, this is the closest thing we have from the available data). Therefore, we will try to find another fighter that have the same height as the fighter with missing data and use that fighter height as our value.
In some cases, we will find multiple fighters with the same height, but different reach. for the sake of simplicity, we will simply get the first non-NaN value

In [23]:
# fetch first non-NA value of each height
height_ref = fighters_clean.groupby('height')['reach'].median()
height_ref

height
--        70.0
4' 7"     65.0
5' 0"     61.5
5' 1"     62.0
5' 10"    72.0
5' 11"    73.0
5' 2"     63.0
5' 3"     64.0
5' 4"     65.0
5' 5"     66.0
5' 6"     67.0
5' 7"     69.0
5' 8"     70.0
5' 9"     71.0
6' 0"     74.0
6' 1"     75.0
6' 10"     NaN
6' 11"    84.0
6' 2"     75.0
6' 3"     77.0
6' 4"     78.0
6' 5"     79.0
6' 6"     79.0
6' 7"     80.5
6' 8"     80.0
7' 2"      NaN
7' 5"      NaN
Name: reach, dtype: float64

In [24]:
# apply and check the amount of missing data left
fighters_clean['reach'] = fighters_clean['reach'].fillna(fighters_clean['height'].map(height_ref))
fighters_clean['reach'].isna().sum()

np.int64(5)

In [25]:
# with only 5, we can now drop those rows
fighters_clean.dropna(subset=['reach'], inplace=True)
fighters_clean['reach'].isna().sum()

np.int64(0)

In [26]:
# stance
fighters_clean['stance'].unique()

array(['Orthodox', 'Southpaw', 'Switch', nan, 'Open Stance', 'Sideways'],
      dtype=object)

In [27]:
# we will simply fill the missing stance with Open Stance
fighters_clean['stance'].fillna('Open Stance', inplace=True)

C:\Users\arnav\AppData\Local\Temp\ipykernel_89736\2635082852.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fighters_clean['stance'].fillna('Open Stance', inplace=True)


In [32]:
# convert height from inch to cm
def convert_to_cms(X):
    if pd.isna(X) or X is None:  # Fixed: use pd.isna() instead of np.NaN
        return X
    elif isinstance(X, str) and len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [33]:
# we have some missing data that hide under the form of "--"
fighters_clean[fighters_clean['height'] == '--']

,SApM,SLpM,height,reach,record,stance,strAcc,strDef,subAvg,tdAcc,tdAvg,tdDef,weight,born_year
name,,,,,,,,,,,,,,


In [34]:
# after some research, most of them have a height of 5'7". So we will replace these values in
feet = "5' "
inches = '7"'
height = feet + inches
fighters_clean['height'].replace({"--": height}, inplace=True)

In [35]:
fighters_clean['height'] = fighters_clean['height'].apply(convert_to_cms)


In [36]:
# split the record
fighters_clean['record'] = fighters_clean['record'].str.replace(' \(', '-(', regex=True)
fighters_clean[['win', 'lose', 'draw', 'nc']] = fighters_clean['record'].str.split('-', expand=True)

def split_nc(nc):
    return re.findall(r"\d+", nc, re.IGNORECASE)[0]
    
fighters_clean['nc'] = fighters_clean['nc'].apply(lambda x: split_nc(x) if x is not None else 0)
fighters_clean.drop(['record'], axis=1, inplace=True)

fighters_clean.head()

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\arnav\AppData\Local\Temp\ipykernel_89736\1692447462.py:2: SyntaxWarning: invalid escape sequence '\('
  fighters_clean['record'] = fighters_clean['record'].str.replace(' \(', '-(', regex=True)


,SApM,SLpM,height,reach,stance,strAcc,strDef,subAvg,tdAcc,tdAvg,tdDef,weight,born_year,win,lose,draw,nc
name,,,,,,,,,,,,,,,,,
Ivan Erslan,5.17,2.50,187.96,72.0,Orthodox,0.44,0.55,0.0,0.20,0.50,0.64,205,1991,14,5,0,1
Paulo Filho,1.67,1.86,172.72,70.0,Southpaw,0.48,0.66,1.6,0.32,2.40,0.42,185,1978,23,6,3,0
Paul Bradley,4.13,2.26,175.26,71.0,Orthodox,0.32,0.56,0.0,0.16,1.12,1.00,170,1983,23,8,0,2
Dan Ige,3.56,3.64,170.18,71.0,Orthodox,0.45,0.56,0.2,0.25,0.89,0.56,145,1991,19,10,0,0
DaMarques Johnson,3.75,2.11,185.42,75.0,Orthodox,0.37,0.51,0.7,0.55,1.18,0.19,170,1982,15,15,0,0


In [37]:
# deal with missing weight hiding as '--'
# first check out how many are left
len(fighters_clean[fighters_clean['weight'] == '--'])

0

In [38]:
# we can jsut drop them
fighters_clean =  fighters_clean[fighters_clean['weight'] != '--']

In [39]:
# finally convert the relevant rows from string to int
def string_2_int(df, columns):
    for column in columns:
        df[column] = df[column].astype(int)
        
        
string_2_int(fighters_clean, ['win', 'lose', 'draw', 'nc', 'weight'])

In [40]:
fighters_clean.dtypes

SApM         float64
SLpM         float64
height       float64
reach        float64
stance        object
strAcc       float64
strDef       float64
subAvg       float64
tdAcc        float64
tdAvg        float64
tdDef        float64
weight         int64
born_year      int64
win            int64
lose           int64
draw           int64
nc             int64
dtype: object

#### df_cards cleaning

In [41]:
p2d(df_cards, ['f1_sig_strike_per', 'f2_sig_strike_per'])
df_cards

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,f2_td_succeed,fight_date,fights_location,round_format,round_fought,weight_class,winner,winning_method
0,UFC 289: Nunes vs. Aldana,Diana Belbita,0.50,208,2,1,Maria Oliveira,0.38,168,5,2,June 10 2023,"Vancouver, British Columbia, Canada",3,3,Women's Strawweight,Diana Belbita,U-DEC
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,0.68,22,1,0,Bruna Brasil,0.58,93,3,3,April 15 2023,"Kansas City, Missouri, USA",3,2,Women's Strawweight,Denise Gomes,KO/TKO
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,0.56,91,3,0,Lando Vannata,0.37,207,1,0,April 15 2023,"Kansas City, Missouri, USA",3,3,Lightweight,Daniel Zellhuber,U-DEC
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,0.31,16,0,0,Tanner Boser,0.75,40,1,1,April 15 2023,"Kansas City, Missouri, USA",3,1,Light Heavyweight,Ion Cutelaba,KO/TKO
4,UFC Fight Night: Holloway vs. Allen,Azamat Murzakanov,0.48,131,6,1,Dustin Jacoby,0.52,127,5,1,April 15 2023,"Kansas City, Missouri, USA",3,3,Light Heavyweight,Azamat Murzakanov,U-DEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8140,UFC Fight Night: Aspinall vs. Tybura,Paul Craig,0.48,82,2,2,Andre Muniz,0.60,40,2,2,July 22 2023,"London, England, United Kingdom",3,2,Middleweight,Paul Craig,KO/TKO
8141,UFC Fight Night: Aspinall vs. Tybura,Nathaniel Wood,0.56,148,2,1,Andre Fili,0.38,151,2,0,July 22 2023,"London, England, United Kingdom",3,3,Featherweight,Nathaniel Wood,U-DEC
8142,UFC Fight Night: Aspinall vs. Tybura,Fares Ziam,0.36,134,6,0,Jai Herbert,0.53,113,5,1,July 22 2023,"London, England, United Kingdom",3,3,Lightweight,Fares Ziam,U-DEC
8143,UFC Fight Night: Aspinall vs. Tybura,Julija Stoliarenko,0.37,16,0,0,Molly McCann,0.31,29,1,1,July 22 2023,"London, England, United Kingdom",3,1,Women's Flyweight,Julija Stoliarenko,SUB


In [42]:
# reuse split birth year function to get year of fight
def get_fight_year(dof):
    return datetime.strptime(dof, '%B %d %Y').year

df_cards['fight_year'] = df_cards['fight_date'].apply(lambda x: get_fight_year(x))
df_cards.drop(['fight_date'], axis=1, inplace=True)

In [43]:
df_cards

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,f2_td_succeed,fights_location,round_format,round_fought,weight_class,winner,winning_method,fight_year
0,UFC 289: Nunes vs. Aldana,Diana Belbita,0.50,208,2,1,Maria Oliveira,0.38,168,5,2,"Vancouver, British Columbia, Canada",3,3,Women's Strawweight,Diana Belbita,U-DEC,2023
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,0.68,22,1,0,Bruna Brasil,0.58,93,3,3,"Kansas City, Missouri, USA",3,2,Women's Strawweight,Denise Gomes,KO/TKO,2023
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,0.56,91,3,0,Lando Vannata,0.37,207,1,0,"Kansas City, Missouri, USA",3,3,Lightweight,Daniel Zellhuber,U-DEC,2023
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,0.31,16,0,0,Tanner Boser,0.75,40,1,1,"Kansas City, Missouri, USA",3,1,Light Heavyweight,Ion Cutelaba,KO/TKO,2023
4,UFC Fight Night: Holloway vs. Allen,Azamat Murzakanov,0.48,131,6,1,Dustin Jacoby,0.52,127,5,1,"Kansas City, Missouri, USA",3,3,Light Heavyweight,Azamat Murzakanov,U-DEC,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8140,UFC Fight Night: Aspinall vs. Tybura,Paul Craig,0.48,82,2,2,Andre Muniz,0.60,40,2,2,"London, England, United Kingdom",3,2,Middleweight,Paul Craig,KO/TKO,2023
8141,UFC Fight Night: Aspinall vs. Tybura,Nathaniel Wood,0.56,148,2,1,Andre Fili,0.38,151,2,0,"London, England, United Kingdom",3,3,Featherweight,Nathaniel Wood,U-DEC,2023
8142,UFC Fight Night: Aspinall vs. Tybura,Fares Ziam,0.36,134,6,0,Jai Herbert,0.53,113,5,1,"London, England, United Kingdom",3,3,Lightweight,Fares Ziam,U-DEC,2023
8143,UFC Fight Night: Aspinall vs. Tybura,Julija Stoliarenko,0.37,16,0,0,Molly McCann,0.31,29,1,1,"London, England, United Kingdom",3,1,Women's Flyweight,Julija Stoliarenko,SUB,2023


In [44]:
#randomly swap f1 and f2 for half of the dataset so that 50% of f2 are winners
swap_indices = np.random.choice(len(df_cards), size= len(df_cards) //2, replace = False)
df_cards.iloc[swap_indices, [1, 6]] = df_cards.iloc[swap_indices, [6, 1]].values


In [45]:
# quick check
df_cards["winner"] = df_cards["winner"] == df_cards["f1"]
df_cards["winner"] = df_cards["winner"].astype(int)
df_cards["winner"].value_counts()

winner
1    4073
0    4072
Name: count, dtype: int64

In [46]:
# we had to change some names earlier due to duplication, we will do the same here
cards_clean = df_cards.copy()
for col in ['f1', 'f2']:
    cards_clean.loc[(cards_clean[col] == 'Michael McDonald') & 
                    (cards_clean['weight_class'] == 'Bantamweight'), col] = "Michael McDonald 135"
    
    cards_clean.loc[(cards_clean[col] == 'Tony Johnson') & 
                    (cards_clean['weight_class'] == 'Heavyweight'), col] = "Tony Johnson 265"
    
    cards_clean.loc[(cards_clean[col] == 'Joey Gomez') & 
                    (cards_clean['weight_class'] == 'Welterweight'), col] = "Joey Gomez 155"
    
    cards_clean.loc[(cards_clean[col] == 'Bruno Silva') & 
                    (cards_clean['weight_class'] == 'Light Heavyweight'), col] = "Bruno Silva 185"
    
  

In [47]:
cards_clean

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,f2_td_succeed,fights_location,round_format,round_fought,weight_class,winner,winning_method,fight_year
0,UFC 289: Nunes vs. Aldana,Diana Belbita,0.50,208,2,1,Maria Oliveira,0.38,168,5,2,"Vancouver, British Columbia, Canada",3,3,Women's Strawweight,1,U-DEC,2023
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,0.68,22,1,0,Bruna Brasil,0.58,93,3,3,"Kansas City, Missouri, USA",3,2,Women's Strawweight,1,KO/TKO,2023
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,0.56,91,3,0,Lando Vannata,0.37,207,1,0,"Kansas City, Missouri, USA",3,3,Lightweight,1,U-DEC,2023
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,0.31,16,0,0,Tanner Boser,0.75,40,1,1,"Kansas City, Missouri, USA",3,1,Light Heavyweight,1,KO/TKO,2023
4,UFC Fight Night: Holloway vs. Allen,Dustin Jacoby,0.48,131,6,1,Azamat Murzakanov,0.52,127,5,1,"Kansas City, Missouri, USA",3,3,Light Heavyweight,0,U-DEC,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8140,UFC Fight Night: Aspinall vs. Tybura,Andre Muniz,0.48,82,2,2,Paul Craig,0.60,40,2,2,"London, England, United Kingdom",3,2,Middleweight,0,KO/TKO,2023
8141,UFC Fight Night: Aspinall vs. Tybura,Nathaniel Wood,0.56,148,2,1,Andre Fili,0.38,151,2,0,"London, England, United Kingdom",3,3,Featherweight,1,U-DEC,2023
8142,UFC Fight Night: Aspinall vs. Tybura,Fares Ziam,0.36,134,6,0,Jai Herbert,0.53,113,5,1,"London, England, United Kingdom",3,3,Lightweight,1,U-DEC,2023
8143,UFC Fight Night: Aspinall vs. Tybura,Molly McCann,0.37,16,0,0,Julija Stoliarenko,0.31,29,1,1,"London, England, United Kingdom",3,1,Women's Flyweight,0,SUB,2023


In [48]:
all_fighters = fighters_clean.index.tolist()
all_fighters[:10]

['Ivan Erslan',
 'Paulo Filho',
 'Paul Bradley',
 'Dan Ige',
 'DaMarques Johnson',
 'Antonio Banuelos',
 'Ramsey Nijem',
 'Journey Newson',
 'Eduardo Neves',
 'Alex Nicholson']

In [49]:
# drop the fights that don't have the figther in the fighters_clean df
cards_clean = cards_clean.loc[(cards_clean["f1"].isin(all_fighters)) &
                              (cards_clean["f2"].isin(all_fighters))]
cards_clean.reset_index(inplace=True, drop=True)

In [50]:
cards_clean.head()

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,f2_td_succeed,fights_location,round_format,round_fought,weight_class,winner,winning_method,fight_year
0,UFC 289: Nunes vs. Aldana,Diana Belbita,0.50,208,2,1,Maria Oliveira,0.38,168,5,2,"Vancouver, British Columbia, Canada",3,3,Women's Strawweight,1,U-DEC,2023
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,0.68,22,1,0,Bruna Brasil,0.58,93,3,3,"Kansas City, Missouri, USA",3,2,Women's Strawweight,1,KO/TKO,2023
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,0.56,91,3,0,Lando Vannata,0.37,207,1,0,"Kansas City, Missouri, USA",3,3,Lightweight,1,U-DEC,2023
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,0.31,16,0,0,Tanner Boser,0.75,40,1,1,"Kansas City, Missouri, USA",3,1,Light Heavyweight,1,KO/TKO,2023
4,UFC Fight Night: Holloway vs. Allen,Dustin Jacoby,0.48,131,6,1,Azamat Murzakanov,0.52,127,5,1,"Kansas City, Missouri, USA",3,3,Light Heavyweight,0,U-DEC,2023


In [51]:
print("we had {} cards initially. After clean up: {} cards".format(len(df_cards), len(cards_clean)))

we had 8145 cards initially. After clean up: 7950 cards


In [52]:
# get data of fighter 1 and fighter 2
f1_data = fighters_clean.loc[cards_clean['f1']]
f1_data = f1_data.add_suffix('_f1')
f2_data = fighters_clean.loc[cards_clean['f2']]
f2_data = f2_data.add_suffix('_f2')

In [53]:
f1_data.head()

,SApM_f1,SLpM_f1,height_f1,reach_f1,stance_f1,strAcc_f1,strDef_f1,subAvg_f1,tdAcc_f1,tdAvg_f1,tdDef_f1,weight_f1,born_year_f1,win_f1,lose_f1,draw_f1,nc_f1
name,,,,,,,,,,,,,,,,,
Diana Belbita,6.55,6.45,170.18,68.0,Orthodox,0.41,0.51,0.0,0.62,0.86,0.64,125,1996,15,10,0,0
Denise Gomes,3.04,4.59,157.48,63.0,Orthodox,0.50,0.51,0.8,0.36,1.19,0.67,115,1999,11,3,0,0
Daniel Zellhuber,5.91,5.83,185.42,77.0,Switch,0.39,0.56,0.1,0.25,0.15,0.94,155,1999,15,3,0,0
Ion Cutelaba,3.34,4.26,185.42,75.0,Southpaw,0.43,0.47,0.1,0.49,3.77,0.75,205,1993,19,11,1,1
Dustin Jacoby,4.02,5.37,190.50,76.0,Orthodox,0.47,0.57,0.0,0.25,0.33,0.62,205,1988,21,9,1,0


In [54]:
f2_data.head()

,SApM_f2,SLpM_f2,height_f2,reach_f2,stance_f2,strAcc_f2,strDef_f2,subAvg_f2,tdAcc_f2,tdAvg_f2,tdDef_f2,weight_f2,born_year_f2,win_f2,lose_f2,draw_f2,nc_f2
name,,,,,,,,,,,,,,,,,
Maria Oliveira,5.76,5.03,167.64,69.0,Orthodox,0.38,0.44,0.0,0.44,0.95,0.47,115,1996,13,7,0,0
Bruna Brasil,4.14,2.77,167.64,65.0,Orthodox,0.59,0.48,0.0,0.66,1.55,0.54,115,1993,10,5,1,0
Lando Vannata,4.76,4.52,175.26,71.0,Orthodox,0.45,0.55,0.3,0.37,1.09,0.69,155,1992,12,7,2,0
Tanner Boser,2.75,4.70,187.96,75.0,Orthodox,0.52,0.58,0.0,0.00,0.00,0.63,205,1991,21,10,1,0
Azamat Murzakanov,2.86,4.93,177.80,71.0,Southpaw,0.57,0.61,0.0,0.15,0.58,0.83,205,1989,15,0,0,0


In [55]:
# join the 2 dataframe
f1_data.reset_index(inplace=True, drop=True)
f2_data.reset_index(inplace=True, drop=True)
final_df = pd.concat([cards_clean, f1_data, f2_data], axis=1, sort=False)

In [56]:
final_df['f1_age_when_fight'] = final_df['fight_year'] - final_df['born_year_f1']
final_df['f2_age_when_fight'] = final_df['fight_year'] - final_df['born_year_f2']


In [57]:
final_df.head()

,card_name,f1,f1_sig_strike_per,f1_sig_strike_total,f1_td_attempt,f1_td_succeed,f2,f2_sig_strike_per,f2_sig_strike_total,f2_td_attempt,...,tdAvg_f2,tdDef_f2,weight_f2,born_year_f2,win_f2,lose_f2,draw_f2,nc_f2,f1_age_when_fight,f2_age_when_fight
0,UFC 289: Nunes vs. Aldana,Diana Belbita,0.50,208.0,2.0,1.0,Maria Oliveira,0.38,168.0,5.0,...,0.95,0.47,115.0,1996.0,13.0,7.0,0.0,0.0,27.0,27.0
1,UFC Fight Night: Holloway vs. Allen,Denise Gomes,0.68,22.0,1.0,0.0,Bruna Brasil,0.58,93.0,3.0,...,1.55,0.54,115.0,1993.0,10.0,5.0,1.0,0.0,24.0,30.0
2,UFC Fight Night: Holloway vs. Allen,Daniel Zellhuber,0.56,91.0,3.0,0.0,Lando Vannata,0.37,207.0,1.0,...,1.09,0.69,155.0,1992.0,12.0,7.0,2.0,0.0,24.0,31.0
3,UFC Fight Night: Holloway vs. Allen,Ion Cutelaba,0.31,16.0,0.0,0.0,Tanner Boser,0.75,40.0,1.0,...,0.00,0.63,205.0,1991.0,21.0,10.0,1.0,0.0,30.0,32.0
4,UFC Fight Night: Holloway vs. Allen,Dustin Jacoby,0.48,131.0,6.0,1.0,Azamat Murzakanov,0.52,127.0,5.0,...,0.58,0.83,205.0,1989.0,15.0,0.0,0.0,0.0,35.0,34.0


In [58]:
# last checks
print(final_df.isna().sum())
print(final_df.dtypes)

card_name              15
f1                     15
f1_sig_strike_per      15
f1_sig_strike_total    15
f1_td_attempt          15
f1_td_succeed          15
f2                     15
f2_sig_strike_per      15
f2_sig_strike_total    15
f2_td_attempt          15
f2_td_succeed          15
fights_location        15
round_format           15
round_fought           15
weight_class           15
winner                 15
winning_method         15
fight_year             15
SApM_f1                 0
SLpM_f1                 0
height_f1               0
reach_f1                0
stance_f1               0
strAcc_f1               0
strDef_f1               0
subAvg_f1               0
tdAcc_f1                0
tdAvg_f1                0
tdDef_f1                0
weight_f1               0
born_year_f1            0
win_f1                  0
lose_f1                 0
draw_f1                 0
nc_f1                   0
SApM_f2                 3
SLpM_f2                 3
height_f2               3
reach_f2    

In [59]:
final_df.to_csv('cleaned_dataset.csv', index=False)